# Section 3.4 — Analytical approximation methods

This notebook contains the code examples from [Section 3.4 Analytical approximation methods]() of the **No Bullshit Guide to Statistics**.

#### Notebook setup

In [1]:
# load Python modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Figures setup
sns.set_theme(
    context="paper",
    style="whitegrid",
    palette="colorblind",
    rc={'figure.figsize': (7,4)},
)

%config InlineBackend.figure_format = 'retina'

In [3]:
# set random seed for repeatability
np.random.seed(42)

In [4]:
# load data for two groups
data = pd.read_csv('../stats_overview/data/employee_lifetime_values.csv')

## Approach 2: Analytical approximations for hypothesis testing

We'll now look at another approach for answering Question 1:
using and analytical approximation,
which is the way normally taught in STATS 101 courses.
How likely or unlikely is the observed difference $d=130$ under the null hypothesis?

- Analytical approximations are math models for describing the sampling distribution under $H_0$
  - Real sampling distributions: obtained by repeated sampling from $H_0$
  - Analytical approximation: probability model based on estimated parameters
- Based on this assumption we can use the theoretical model for the difference between group means
  that we developed earlier, we can obtain a **closed form expression** for the sampling distribution of $D$.
- In particular, the probability model for the two groups under $H_0$ are:
  $$ 
       H_0: \qquad X_S = \mathcal{N}(\color{red}{\mu_0}, \sigma_S)
       \quad \textrm{and} \quad
       X_{NS} = \mathcal{N}(\color{red}{\mu_0}, \sigma_{NS}), \quad
  $$
  from which we can derive the model for $D = \overline{\mathbf{X}}_S - \overline{\mathbf{X}}_{NS}$:
  $$ 
     D  \sim \mathcal{N}\!\left( \color{red}{0}, \  \sqrt{ \tfrac{\sigma^2_S}{n_S} + \tfrac{\sigma^2_{NS}}{n_{NS}} } \right)
  $$
  In words, the sampling distribution of the difference between group means is
  normally distributed with mean $\mu_D = 0$ and standard deviation $\sigma_D$,
  which depends on the variance of the two groups $\sigma^2_S$ and $\sigma^2_{NS}$.
  Recall we obtained this expression earlier when we discussed difference of means between groups A and B.
- However, the population variances $\sigma^2_S$ and $\sigma^2_{NS}$ are unknown,
  and we only have the estimated variances $s_S^2$ and $s_{NS}^2$,
  which we calculated from the sample.
- That's OK though, since the sample variances are estimates for the population variances.
  There are two common ways to obtain an approximation for $\sigma^2_D$:
  - Pooled variance: $\sigma^2_D \approx s^2_p =  \frac{(n_S-1)s_S^2 \; + \; (n_{NS}-1)s_{NS}^2}{n_S + n_{NS} - 2}$
    (takes advantage of assumption that both samples come from the same population under $H_0$)
  - Unpooled variance: $\sigma^2_D \approx s^2_u = \tfrac{s^2_S}{n_S} + \tfrac{s^2_{NS}}{n_{NS}}$
    (follows from the general rule of probability theory)
- NEW CONCEPT: **Student's $t$-distribution** is a model for $D$ which takes into account
  we are using $s_S^2$ and $s_{NS}^2$ instead of $\sigma_S^2$ and $\sigma_{NS}^2$.
- NEW CONCEPT: **degrees of freedom**, denoted `df` in code or $\nu$ (Greek letter *nu*) in equations,
  is the parameter of Student's $t$-distribution dependent on the sample size used to estimate quantities.


### Student's t-test (pooled variance)

[Student's $t$-test for comparison of the difference between two groups means](https://statkat.com/stattest.php?&t=9) is a procedure that makes use of the pooled variance $s^2_p$. 
Recall $H_0$ states there is no difference between the two groups.
This means we can think of $s_S^2$ and $s_{NS}^2$ as two independent estimates of the population variance,
so we can combine them (pool them together) to obtain an estimate $s^2_p$.

#### Black-box approach
The `scipy.stats` function `ttest_ind` will perform all the steps of Student's $t$-test procedure,
without the need for us to understand the details.

In [5]:
from scipy.stats import ttest_ind

# extract data for two groups
xS = data[data["group"]=="S"]['ELV']
xNS = data[data["group"]=="NS"]['ELV']

# run the complete t-test procedure for ind-ependent samples:
result = ttest_ind(xS, xNS)
result.pvalue

0.046999086677830995

The $p$-value is less than 0.05 so our decision is to **reject the null hypothesis**.

#### Student's t-test under the hood

The computations hidden behind the function `ttest_ind` involve a six step procedure that makes use of the pooled variance $s^2_p$.

In [6]:
from statistics import stdev
from scipy.stats import t as tdist

# 1. calculate the mean in each group
meanS, meanNS = np.mean(xS), np.mean(xNS)

# 2. calculate d, the observed difference between means
d = meanS - meanNS

# 3. calculate the standard deviations in each group
stdS, stdNS = stdev(xS), stdev(xNS)
nS, nNS = len(xS), len(xNS)

# 4. compute the pooled variance and standard error
var_pooled = ((nS-1)*stdS**2 + (nNS-1)*stdNS**2)/(nS + nNS - 2)
std_pooled = np.sqrt(var_pooled)
std_err = std_pooled * np.sqrt(1/nS + 1/nNS)

# 5. compute the value of the t-statistic
tstat = d / std_err

# 6. obtain the p-value for the t-statistic from a 
#    t-distribution with 31+30-2 = 59 degrees of freedom
df = nS + nNS - 2
pvalue = 2 * tdist(df).cdf(-abs(tstat))  # 2* because two-sided

pvalue

0.04699908667783097

#### Welch's t-test (unpooled variances)

An [alternative $t$-test procedure](https://statkat.com/stattest.php?&t=9) that doesn't assume the variances in the two groups are equal.

In [7]:
result2 = ttest_ind(xS, xNS, equal_var=False)
result2.pvalue

0.046579019827041344

Welch's $t$-test differs only in steps 4 through 6 as shown below:

In [8]:
# 4'. compute the unpooled standard deviation of D
stdD = np.sqrt(stdS**2/nS + stdNS**2/nNS)

# 5'. compute the value of the t-statistic
tstat = d / stdD

# 6'. obtain the p-value from a t-distribution from
# this crazy formula for the degrees of freedom:
df = (stdS**2/nS + stdNS**2/nNS)**2 / \
    ((stdS**2/nS)**2/(nS-1) + (stdNS**2/nNS)**2/(nNS-1) )
pvalue = 2 * tdist(df).cdf(-abs(tstat))  # 2* because two-sided

pvalue

0.04657901982704139

### Summary of Question 1

We saw two ways to answer Question 1 (is there really a difference between group means) and obtain the $p$-value.
We interpreted the small $p$-values as evidence that the observed difference, $d=130$, is unlikely to be due to chance under $H_0$, so we rejected the null hypothesis.
Note this whole procedure is just a sanity check—we haven't touched the alternative hypothesis at all yet,
and for all we know the stats training could have the effect of decreasing ELV!


## Estimating the effect size

- Question 2 of Amy's statistical investigation is to estimate the difference in ELV gained by stats training.
- NEW CONCEPT: **effect size** is a measure of difference between intervention and control groups.
- We assume the data of **Group S** and **Group NS** come from different populations with means $\mu_S$ and $\mu_{NS}$.
- We're interested in estimating the difference between population means, denoted $\Delta = \mu_S - \mu_{NS}$.
- By analyzing the sample, we have obtained an estimate $d=130$ for the unknown $\Delta$,
  but we know our data contains lots of variability, so we know our estimate might be off.
- We want an answer to Question 2 (What is the estimated difference between group means?)
  that takes into account the variability of the data.
- NEW CONCEPT: **confidence interval** is a way to describe a range of values for an estimate
  that takes into account the variability of the data.
- We want to provide an answer to Question 2 in the form of a confidence interval that tells
  us a range of values where we believe the true value of $\Delta$ falls.
- Similar to how we showed two approaches for hypothesis testing,
  we'll work on effect size estimation using two approaches: bootstrap estimation and analytical approximation methods.


### Approach 2: Confidence intervals using analytical approximations

  - Assumption: the variance of the two populations is the same (or approximately equal)
  - Using the theoretical model for the populations,
    we can obtain a formula for CI of effect size $\Delta$:
    $$
    \textrm{CI}_{(1-\alpha)}
    = \left[ d - t^*\!\cdot\!\sigma_D, \, 
             d + t^*\!\cdot\!\sigma_D
      \right].
    $$
    The confidence interval is centred at $d$,
    with width proportional to the standard deviation $\sigma_D$.
    The constant $t^*$ denotes the value of the inverse CDF of Student's $t$-distribution
    with appropriate number of degrees of freedom `df` evaluated at $1-\frac{\alpha}{2}$.
    For a 90% confidence interval, we choose $\alpha=0.10$,
    which gives $(1-\frac{\alpha}{2}) = 0.95$, $t^* = F_{T_{\textrm{df}}}^{-1}\left(0.95\right)$.
  - We can use the two different analytical approximations to obtain a formula for $\sigma_D$
    just as we did in the hypothesis testing:
    - Pooled variance: $\sigma^2_p =  \frac{(n_S-1)s_S^2 + (n_{NS}-1)s_{NS}^2}{n_S + n_{NS} - 2}$,
      and `df` = $n_S + n_{NS} -2$
    - Unpooled variance: $\sigma^2_u = \tfrac{s^2_A}{n_A} + \tfrac{s^2_B}{n_B}$, and `df` = [...](https://en.wikipedia.org/wiki/Student%27s_t-test#Equal_or_unequal_sample_sizes,_unequal_variances_(sX1_%3E_2sX2_or_sX2_%3E_2sX1))

#### Using pooled variance

The calculations are similar to Student's t-test for hypothesis testing.

In [9]:
from scipy.stats import t

d = np.mean(xS) - np.mean(xNS)

nS, nNS = len(xS), len(xNS)
stdS, stdNS = stdev(xS), stdev(xNS)
var_pooled = ((nS-1)*stdS**2 + (nNS-1)*stdNS**2)/(nS + nNS - 2)
std_pooled = np.sqrt(var_pooled)
std_err = std_pooled * np.sqrt(1/nS + 1/nNS)

df = nS + nNS - 2

# for 90% confidence interval, need 10% in tails
alpha = 0.10

# now use inverse-CDF of Students t-distribution
tstar = abs(t(df).ppf(alpha/2))

CI_tpooled = [d - tstar*std_err, d + tstar*std_err]
CI_tpooled

[22.925130302086643, 237.12228905275202]

#### Using unpooled variance

The calculations are similar to the Welch's t-test for hypothesis testing.

In [10]:
d = np.mean(xS) - np.mean(xNS)

nS, nNS = len(xS), len(xNS)
stdS, stdNS = stdev(xS), stdev(xNS)
stdD = np.sqrt(stdS**2/nS + stdNS**2/nNS)

df = (stdS**2/nS + stdNS**2/nNS)**2 / \
    ((stdS**2/nS)**2/(nS-1) + (stdNS**2/nNS)**2/(nNS-1) )

# for 90% confidence interval, need 10% in tails
alpha = 0.10

# now use inverse-CDF of Students t-distribution
tstar = abs(t(df).ppf(alpha/2))

CI_tunpooled = [d - tstar*stdD, d + tstar*stdD]
CI_tunpooled

[23.14219839967336, 236.9052209551653]


## Comparison of resampling methods and analytical approximations

In this notebook we saw two different approaches for doing statistical analysis: resampling methods and analytical approximations. This is a general pattern in statistics where there is not only one correct answer: multiple approaches to data analysis are valid, and you need to think about the specifics of each situation. We'll learn about both approaches in the book.

Analytical approximations are currently taught in most stats courses (STAT 101). Historically, analytical approximations have been used more widely because they require only simple arithmetic calculations: statistic practitioners (scientists, engineers, etc.) simply need to compute sample statistics, plug them into a formula, and obtain a $p$-value. This convenience comes at the cost of numerous assumptions about the data distribution, which often don't hold in practice (e.g. assuming population is normal, when it isn't).

In recent years, resampling methods like the permutation test and bootstrap estimation are becoming more popular, widely used in industry, and increasingly also taught at universities (see this [blog post](https://minireference.com/blog/fixing-the-introductory-statistics-curriculum/) about the *modern statistics* curriculum). **The main advantage so resampling methods is that they require less modelling assumptions.** Procedures like the permutation test can be applied broadly to any scenario where two groups are being compared, and don't require developing specific formulas for different cases. Resampling methods are easier to understand since the they are directly related to the sampling distribution, and there are no formulas to memorize.

Understanding resampling methods requires some basic familiarity with programming, but the skills required are not advanced: knowledge of variables, expressions, and basic `for` loop is sufficient. If you were able to follow the code examples described above (see `resample_under_H0`, `permutation_test`, and `bootstrap_stat`), then you've already **seen all the code you will need for the entire book!**  I've prepared a [python tutorial](https://nobsstats.com/tutorials/python_tutorial.html) to make readers with no prior experience with Python will be able to quickly pick up the syntax.